# Complex bonds

You can run the cells below directly in LUSID's JupyterHub.

The `%%luminesce` is a magic command which passes the cell query string to Lumipy,
which then returns a DataFrame.
    

#### Step 01: Fixed schedule view

In [ ]:
%%luminesce

/*
    
    Fixed Schedule View Creation


*/

-- Fixed Schedule View Input Table
@fixedTableExample = select
'AssetID' as [AssetID],
'2020-01-01T00:00:00+00:00' as [StartDate],
'2022-01-01T00:00:00+00:00' as [MaturityDate],
'GBP' as [Currency],
'3M' as [PaymentFrequency],
'ActAct' as [DayCountConvention],
'None' as [RollConvention],
'GBP' as [PaymentCalendars],
'GBP"]' as [ResetCalendars],
3 as [SettleDays],
0 as [ResetDays],
true as [LeapDaysIncluded],
1 as [Notional],
0.0123 as [CouponRate],
'GBP' as [PaymentCurrency],
'LongFront' as [StubType];

@createFixedScheduleJsonView =  use Sys.Admin.SetupView with @fixedTableExample
--provider=Schedules.Fixed_Schedule
--description="Outputs a table with two columns containing the asset ID and the formatted fixed schedule json"
--parameters
FixedTable,Table,@fixedTableExample,true
----

@fixedTable = select * from #PARAMETERVALUE(FixedTable);

select
ft.[AssetID],
json_object(
    'startDate', ft.[StartDate],
    'maturityDate', ft.[MaturityDate],
    'flowConventions', json_object(
        'currency', ft.[Currency],
        'paymentFrequency', ft.[PaymentFrequency],
        'dayCountConvention', ft.[DayCountConvention],
        'rollConvention', ft.[RollConvention],
        'paymentCalendars', json_array(ft.[PaymentCalendars]),
        'resetCalendars', json_array(ft.[ResetCalendars]),
        'settleDays', cast(ft.[SettleDays] as int),
        'resetDays', cast(ft.[ResetDays] as int),
        'leapDaysInclueded', ft.[LeapDaysIncluded]
    ),
    'notional', cast(ft.[Notional] as double),
    'couponRate', cast(ft.[CouponRate] as decimal),
    'paymentCurrency', ft.[PaymentCurrency],
    'stubType', ft.[StubType],
    'scheduleType', 'Fixed'
) as JsonString
from @fixedTable ft;

enduse;


#### Step 02: Step schedule view

In [ ]:
%%luminesce

/*
===========================================
        Step Schedule View Creation
===========================================
*/

-- Step Schedule View Input Table
@stepsTableExample = select
'AssetID' as [AssetID],
'2021-09-15T00:00:00+00:00' as [Date],
1 as [Quantity];

@createStepScheduleJsonView =  use Sys.Admin.SetupView with @stepsTableExample
--provider=Schedules.Step_Schedule
--description="Outputs a table with two columns containing the asset ID and the formatted step schedule json"
--parameters
LevelType,Text,Absolute,true
StepScheduleType,Text,Coupon,true
StepsTable,Table,@stepsTableExample,true
AssetFilter, Text, AssetFilter, true
----
@@levelType = select #PARAMETERVALUE(LevelType);
@@stepScheduleType =  select #PARAMETERVALUE(StepScheduleType);
@@assetFilter = select #PARAMETERVALUE(AssetFilter);
@stepsTable = select * from #PARAMETERVALUE(StepsTable) where AssetID = @@assetFilter;

select
st.[AssetID],
json_object(
    'steps', json_group_array(
        json_object(
            'date', st.[Date],
            'quantity', cast(st.[Quantity] as double)
        )
    ),
    'levelType', @@levelType,
    'stepScheduleType', @@stepScheduleType,
    'scheduleType', 'Step'
) as JsonString
from @stepsTable st;

enduse;



#### Step 03: Upload short front sinking complex bond insturments

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- In this query, we run an ETL process on some complex bonds.
-- 1. First, we load an Excel file of complex bond data from Drive.
-- 2. Next, we transform the core data and create schedule json fields.
-- 3. Then we combine the schedules json fields into a single column.
-- 4. Finally we upload the instrument data into LUSID.
-- ============================================================

----------------------
-- Define constants --
----------------------

@@data = select 'fixed-stubs-sinking-bond.xlsx';
@@file = select 'luminesce-examples';
@@levelType = select 'Absolute';
@@stepScheduleType = select 'Notional';
@@scope = select 'complexBondTesting';


--------------------------------
-- 1. Extract Instrument Data --
--------------------------------

-- Extract bond data from LUSID Drive

@extractCmplxBondAssetData = use Drive.Excel with @@data, @@file
--file={@@file}
--folderFilter={@@data}
--worksheet=bond-data
enduse;

-- Extract step schedule data from LUSID Drive

@extractCmplxBondStepsData = use Drive.Excel with @@data, @@file
--file={@@file}
--folderFilter={@@data}
--worksheet=steps-data
enduse;


---------------------------------
-- 2. Transform data using SQL --
---------------------------------

-- Transform core complex bond fields

@coreCmplxBondData = select
[Name] as [DisplayName],
[ISIN] as [Isin],
[AssetID] as [ClientInternal],
'Standard' as [CalculationType]
from @extractCmplxBondAssetData;

-- Transform front fixed schedule data

@formatFrontStubFixedScheduleData = select
[AssetID] as [AssetID],
[Interest Start Date] as [StartDate],
[First Coupon Date] as [Maturitydate],
[Currency] as [Currency],
[DCC] as [DayCountConvention],
[Payment Frequency] as [PaymentFrequency],
'None' as [RollConvention],
[Currency] as[PaymentCalendars],
[Currency] as [ResetCalendars],
[Settle Days] as [SettleDays],
0 as [ResetDays],
true as [LeapDaysIncluded],
[Notional] as [Notional],
[Coupon] as [CouponRate],
[Currency] as [PaymentCurrency],
[Front Stub Type] as [StubType]
from @extractCmplxBondAssetData;

-- Add front fixed schedule JSON fields to core fields

@addFrontStubSchedule = select d.*, d.ClientInternal, results.*
from @coreCmplxBondData d
outer apply (
    select JsonString as [FixedScheduleJSONFront] from Schedules.Fixed_schedule fs where
fs.AssetID = d.ClientInternal and fs.FixedTable = @formatFrontStubFixedScheduleData and d.ClientInternal = fs.AssetID
) results ;

-- Transform back fixed schedule data

@formatBackStubFixedScheduleData = select
[AssetID] as [AssetID],
[First Coupon Date] as [StartDate],
[Maturity Date] as [Maturitydate],
[Currency] as [Currency],
[DCC] as [DayCountConvention],
[Payment Frequency] as [PaymentFrequency],
'None' as [RollConvention],
[Currency] as[PaymentCalendars],
[Currency] as [ResetCalendars],
[Settle Days] as [SettleDays],
0 as [ResetDays],
true as [LeapDaysIncluded],
[Notional] as [Notional],
[Coupon] as [CouponRate],
[Currency] as [PaymentCurrency],
[Back Stub Type] as [StubType]
from @extractCmplxBondAssetData;

-- Add back fixed schedule JSON fields to core fields

@addBackStubSchedule = select d.*, d.ClientInternal, results.*
from @addFrontStubSchedule d
outer apply (
    select JsonString as [FixedScheduleJSONBack] from Schedules.Fixed_schedule fs where
fs.AssetID = d.ClientInternal and fs.FixedTable = @formatBackStubFixedScheduleData and d.ClientInternal = fs.AssetID
) results ;

-- Transform step schedule data

@formatStepScheduleData = select
[AssetID] as [AssetID],
[Date] as [Date],
[Value] as [Quantity]
from @extractCmplxBondStepsData;

-- Add step schedule JSON fields to core fields

@addStepSchedule = select d.*, d.ClientInternal, results.*
from @addBackStubSchedule d
outer apply (
    select JsonString as [StepScheduleJSON] from Schedules.Step_schedule ss where
ss.StepsTable = @formatStepScheduleData and ss.LevelType = @@levelType and ss.StepScheduleType = @@stepScheduleType and ss.AssetFilter = d.ClientInternal and d.ClientInternal = ss.AssetID
) results ;

-------------------------------
-- 3. Combine schedules JSON --
-------------------------------

-- Format full complex bond data with combined schedules list column

@complexBondDataToLoad = select
[DisplayName],
[Isin],
[ClientInternal],
[CalculationType],
json_array(
    json(FixedScheduleJSONFront), json(FixedScheduleJSONBack), json(StepScheduleJSON)
) as [SchedulesJson]
from @addStepSchedule;

-----------------------------------------
-- 4. Load formatted complex bond data --
-----------------------------------------

@load = select * from Lusid.Instrument.ComplexBond.Writer where
ToWrite = @complexBondDataToLoad;

select * from @load;



#### Step 1: Fixed schedule view

In [ ]:
%%luminesce

/*
============================================
        Fixed Schedule View Creation
============================================
*/

-- Fixed Schedule View Input Table
@fixedTableExample = select
'AssetID' as [AssetID],
'2020-01-01T00:00:00+00:00' as [StartDate],
'2022-01-01T00:00:00+00:00' as [MaturityDate],
'GBP' as [Currency],
'3M' as [PaymentFrequency],
'ActAct' as [DayCountConvention],
'None' as [RollConvention],
'GBP' as [PaymentCalendars],
'GBP"]' as [ResetCalendars],
3 as [SettleDays],
0 as [ResetDays],
true as [LeapDaysIncluded],
1 as [Notional],
0.0123 as [CouponRate],
'GBP' as [PaymentCurrency],
'LongFront' as [StubType];

@createFixedScheduleJsonView =  use Sys.Admin.SetupView with @fixedTableExample
--provider=Schedules.Fixed_Schedule
--description="Outputs a table with two columns containing the asset ID and the formatted fixed schedule json"
--parameters
FixedTable,Table,@fixedTableExample,true
----

@fixedTable = select * from #PARAMETERVALUE(FixedTable);

select
ft.[AssetID],
json_object(
    'startDate', ft.[StartDate],
    'maturityDate', ft.[MaturityDate],
    'flowConventions', json_object(
        'currency', ft.[Currency],
        'paymentFrequency', ft.[PaymentFrequency],
        'dayCountConvention', ft.[DayCountConvention],
        'rollConvention', ft.[RollConvention],
        'paymentCalendars', json_array(ft.[PaymentCalendars]),
        'resetCalendars', json_array(ft.[ResetCalendars]),
        'settleDays', cast(ft.[SettleDays] as int),
        'resetDays', cast(ft.[ResetDays] as int),
        'leapDaysInclueded', ft.[LeapDaysIncluded]
    ),
    'notional', cast(ft.[Notional] as double),
    'couponRate', cast(ft.[CouponRate] as decimal),
    'paymentCurrency', ft.[PaymentCurrency],
    'stubType', ft.[StubType],
    'scheduleType', 'Fixed'
) as JsonString
from @fixedTable ft;

enduse;


#### Step 2: Step schedule view

In [ ]:
%%luminesce

/*
===========================================
        Step Schedule View Creation
===========================================
*/

-- Step Schedule View Input Table
@stepsTableExample = select
'AssetID' as [AssetID],
'2021-09-15T00:00:00+00:00' as [Date],
1 as [Quantity];

@createStepScheduleJsonView =  use Sys.Admin.SetupView with @stepsTableExample
--provider=Schedules.Step_Schedule
--description="Outputs a table with two columns containing the asset ID and the formatted step schedule json"
--parameters
LevelType,Text,Absolute,true
StepScheduleType,Text,Coupon,true
StepsTable,Table,@stepsTableExample,true
AssetFilter, Text, AssetFilter, true
----
@@levelType = select #PARAMETERVALUE(LevelType);
@@stepScheduleType =  select #PARAMETERVALUE(StepScheduleType);
@@assetFilter = select #PARAMETERVALUE(AssetFilter);
@stepsTable = select * from #PARAMETERVALUE(StepsTable) where AssetID = @@assetFilter;

select
st.[AssetID],
json_object(
    'steps', json_group_array(
        json_object(
            'date', st.[Date],
            'quantity', cast(st.[Quantity] as double)
        )
    ),
    'levelType', @@levelType,
    'stepScheduleType', @@stepScheduleType,
    'scheduleType', 'Step'
) as JsonString
from @stepsTable st;

enduse;



#### Step 3: Upload short front sinking complex bond insturments

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- In this query, we run an ETL process on some complex bonds.
-- 1. First, we load an Excel file of complex bond data from Drive.
-- 2. Next, we transform the core data and create schedule json fields.
-- 3. Then we combine the schedules json fields into a single column.
-- 4. Finally we upload the instrument data into LUSID.
-- ============================================================

----------------------
-- Define constants --
----------------------

@@data = select 'fixed-stubs-sinking-bond.xlsx';
@@file = select 'luminesce-examples';
@@levelType = select 'Absolute';
@@stepScheduleType = select 'Notional';
@@scope = select 'complexBondTesting';


--------------------------------
-- 1. Extract Instrument Data --
--------------------------------

-- Extract bond data from LUSID Drive

@extractCmplxBondAssetData = use Drive.Excel with @@data, @@file
--file={@@file}
--folderFilter={@@data}
--worksheet=bond-data
enduse;

-- Extract step schedule data from LUSID Drive

@extractCmplxBondStepsData = use Drive.Excel with @@data, @@file
--file={@@file}
--folderFilter={@@data}
--worksheet=steps-data
enduse;


---------------------------------
-- 2. Transform data using SQL --
---------------------------------

-- Transform core complex bond fields

@coreCmplxBondData = select
[Name] as [DisplayName],
[ISIN] as [Isin],
[AssetID] as [ClientInternal],
'Standard' as [CalculationType]
from @extractCmplxBondAssetData;

-- Transform front fixed schedule data

@formatFrontStubFixedScheduleData = select
[AssetID] as [AssetID],
[Interest Start Date] as [StartDate],
[First Coupon Date] as [Maturitydate],
[Currency] as [Currency],
[DCC] as [DayCountConvention],
[Payment Frequency] as [PaymentFrequency],
'None' as [RollConvention],
[Currency] as[PaymentCalendars],
[Currency] as [ResetCalendars],
[Settle Days] as [SettleDays],
0 as [ResetDays],
true as [LeapDaysIncluded],
[Notional] as [Notional],
[Coupon] as [CouponRate],
[Currency] as [PaymentCurrency],
[Front Stub Type] as [StubType]
from @extractCmplxBondAssetData;

-- Add front fixed schedule JSON fields to core fields

@addFrontStubSchedule = select d.*, d.ClientInternal, results.*
from @coreCmplxBondData d
outer apply (
    select JsonString as [FixedScheduleJSONFront] from Schedules.Fixed_schedule fs where
fs.AssetID = d.ClientInternal and fs.FixedTable = @formatFrontStubFixedScheduleData and d.ClientInternal = fs.AssetID
) results ;

-- Transform back fixed schedule data

@formatBackStubFixedScheduleData = select
[AssetID] as [AssetID],
[First Coupon Date] as [StartDate],
[Maturity Date] as [Maturitydate],
[Currency] as [Currency],
[DCC] as [DayCountConvention],
[Payment Frequency] as [PaymentFrequency],
'None' as [RollConvention],
[Currency] as[PaymentCalendars],
[Currency] as [ResetCalendars],
[Settle Days] as [SettleDays],
0 as [ResetDays],
true as [LeapDaysIncluded],
[Notional] as [Notional],
[Coupon] as [CouponRate],
[Currency] as [PaymentCurrency],
[Back Stub Type] as [StubType]
from @extractCmplxBondAssetData;

-- Add back fixed schedule JSON fields to core fields

@addBackStubSchedule = select d.*, d.ClientInternal, results.*
from @addFrontStubSchedule d
outer apply (
    select JsonString as [FixedScheduleJSONBack] from Schedules.Fixed_schedule fs where
fs.AssetID = d.ClientInternal and fs.FixedTable = @formatBackStubFixedScheduleData and d.ClientInternal = fs.AssetID
) results ;

-- Transform step schedule data

@formatStepScheduleData = select
[AssetID] as [AssetID],
[Date] as [Date],
[Value] as [Quantity]
from @extractCmplxBondStepsData;

-- Add step schedule JSON fields to core fields

@addStepSchedule = select d.*, d.ClientInternal, results.*
from @addBackStubSchedule d
outer apply (
    select JsonString as [StepScheduleJSON] from Schedules.Step_schedule ss where
ss.StepsTable = @formatStepScheduleData and ss.LevelType = @@levelType and ss.StepScheduleType = @@stepScheduleType and ss.AssetFilter = d.ClientInternal and d.ClientInternal = ss.AssetID
) results ;

-------------------------------
-- 3. Combine schedules JSON --
-------------------------------

-- Format full complex bond data with combined schedules list column

@complexBondDataToLoad = select
[DisplayName],
[Isin],
[ClientInternal],
[CalculationType],
json_array(
    json(FixedScheduleJSONFront), json(FixedScheduleJSONBack), json(StepScheduleJSON)
) as [SchedulesJson]
from @addStepSchedule;

-----------------------------------------
-- 4. Load formatted complex bond data --
-----------------------------------------

@load = select * from Lusid.Instrument.ComplexBond.Writer where
ToWrite = @complexBondDataToLoad;

select * from @load;

